In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import h5py

PATH = '/storage/home/adz6/group/project/'

In [ ]:
file_path = os.path.join(PATH, 'results/mayfly', '211129_grad_b_frequency_grid_radius_angle.npz')
scores = np.load(file_path)

In [ ]:
list(scores.keys())

In [ ]:
file_path = os.path.join(PATH, 'datasets/data', '211116_grad_b_est.h5')

In [ ]:
h5file = h5py.File(file_path)

In [ ]:
h5file['data'].shape

In [ ]:
h5file['meta'].keys()

In [ ]:
print(np.unique(h5file['meta']['theta_min'][:]))

In [ ]:
print(np.unique(h5file['meta']['x_min'][:]))

# do the matrix multiplication

In [ ]:
data = h5file['data'][:]

In [ ]:
scores = np.matmul(data, data.conjugate().T)

In [ ]:
plt.imshow(abs(scores), interpolation = 'none')

In [ ]:
scores.shape

In [ ]:
scores_diag = abs(np.diag(scores))

In [ ]:
print(scores_diag)

# normalize the data to 10 K noise

In [ ]:
var = 1.38e-23 * 10 * 50 * 250e6

In [ ]:
norm = 1 #/ np.sqrt(var * (abs(data) ** 2).sum(axis=-1) )

In [ ]:
norm

In [ ]:
scores_diag * norm

In [ ]:
norm_scores = scores_diag * norm

# sort scores into a matrix

In [ ]:
radial_positions = np.unique(h5file['meta']['x_min'][:])
pitch_angles = np.unique(h5file['meta']['theta_min'][:])

r_grid, pa_grid = np.meshgrid(radial_positions, pitch_angles)
    
sorted_scores = np.zeros(r_grid.size)

for i, pair in enumerate(zip(r_grid.flatten(), pa_grid.flatten())):
    
    #print(np.argwhere(h5file['meta']['x_min'][:] == pair[0]))
    #print(np.argwhere(h5file['meta']['x_min'][:] == pair[1]))
    try:
        index = np.argwhere(np.logical_and(h5file['meta']['x_min'][:] == pair[0], h5file['meta']['theta_min'][:] == pair[1])).squeeze()
    
        sorted_scores[i] = norm_scores[index]
    except BaseException as err:
        print(err)


In [ ]:
plt.imshow(sorted_scores.reshape(r_grid.shape) / (8192 * 60), aspect='auto', interpolation='none')
plt.colorbar()

In [ ]:
r_grid.size

# plot a restricted region of parameter space

In [ ]:
sorted_scores_reshape = sorted_scores.reshape(r_grid.shape)

In [ ]:
select_rads = np.argwhere(np.logical_and(radial_positions >= 0.01, radial_positions <= 0.04)).squeeze()
select_pa = np.argwhere(np.logical_and(pitch_angles >= 85.0, pitch_angles <= 89.0)).squeeze()

In [ ]:
plt.imshow(sorted_scores_reshape[select_pa, :][:, select_rads], aspect='auto', interpolation='none')

In [ ]:
print(np.mean(sorted_scores_reshape[select_pa, :][:, select_rads]))

# save score grid 

In [ ]:
save_path = os.path.join(PATH, 'results/matched_filter/sparse_grid')
save_name = '220211_radial_pitch_angle_matched_filter_score_grid.npz'

np.savez(os.path.join(save_path, save_name), scores = sorted_scores_reshape, r = r_grid, pa = pa_grid)